# Image Classification Using Transfer Learning

This notebook provides general framework for image classification using transfer learning. The user should input the image folder path and choose the model parameters. The user should choose the network among the pre-saved networks in Pytorch. Also the user should change MLP classifier part to be consistent with the labels in the image folder.  

## 1. Imports and data loaders

In [ ]:
#imports
import numpy as np
from glob import glob
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import matplotlib.pyplot as plt                        
%matplotlib inline
import os
import datetime
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pandas as pd

Create data loaders:

In [ ]:
#load data
dataPath="your image folder path"
#define data transforms
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# data
train_data = datasets.ImageFolder(dataPath + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(dataPath+ '/valid', transform=valid_transforms)
test_data = datasets.ImageFolder(dataPath+ '/test', transform=test_transforms)

# loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=28, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

## 2. Model Parameters

Model parameters include: network index, pre-trained network, re-train existing saved model, and learning rate. 

In [ ]:
#model parameters
# do not edit this part
net_dic={0:"alexnet",
        1:"vgg11",
        2:"vgg13",
        3:"vgg16",
        4:"vgg19",
        5:"vgg11_bn",
        6:"vgg13_bn",
        7:"vgg16_bn",
        8:"vgg19_bn",
        9:"resnet18",
        10:"resnet34",
        11:"resnet50",
        12:"resnet101",
        13:"resnet152",
        14:"densenet121",
        15:"densenet169",
        16:"densenet201",
        17:"densenet161",
        18:"inception_v3",
}
FC=[9216,25088,25088,25088,25088,25088,25088,25088,25088,512,512,2048,2048,2048,1024,1664,1920,2208,2048]
# you can change settins in this part
net_index=11 # index of network to be choosen
selected_net=net_dic[net_index]
output_classes=len(train_data.class_to_idx)
full_train=True # full train network or part of it
net_pretrained=False # load pre-trained network or not
retrain=True # re-train saved model
learning_rate=0.00000001
# enter model path and file name where you want to load or save your new model
model_path="Your model full path and name"


## 3. Functions Definitions 

In this section, the functions needed to train and test model are defined.

### 3.1 Testing Model

In [ ]:
#testing model
# returns test accuracy, loss, number of correctly labeled predictions
# input: data loader, model, loss function, device
def test_model( model, loader,criterion, device):
    test_loss = 0.0
    test_count=0.0
    test_size=0.0
    test_accuracy = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            test_size+=len(labels)
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_count+=torch.sum(equals.type(torch.FloatTensor))
        test_accuracy=test_count/test_size
        test_loss=test_loss/len(loader)
    return test_accuracy,test_loss,test_count

### 3.2 Testing and Reporting Model Performance

In [ ]:
#testing and reporting model
# returns test accuracy, and number of correctly labeled predictions,classification report, confusion matrix
# input: data loader, model, loss, device
def test_report(model, loader,device):
    test_count=0.0
    test_size=0
    test_accuracy = 0.0
    predictions=[]
    true_labels=[]
    model.eval()
    with torch.no_grad():
        for inputs,labels in loader:
            test_size+=len(inputs)
            inputs=inputs.to(device)
            labels=labels.to(device)
            Z = model.forward(inputs)
            ps = torch.exp(Z)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_count+=torch.sum(equals.type(torch.FloatTensor))
            true_labels+=(labels.squeeze().tolist())
            predictions+=(top_class.squeeze().tolist())
        test_accuracy=test_count/test_size
        report=classification_report(true_labels, predictions)
        conf_matrix=confusion_matrix(true_labels,predictions)   
    return test_accuracy,test_count,report,conf_matrix

### 3.3 Training Model

In [ ]:
# training model
# imput: number of epochs,model,  train loader, validation loader, ptimizer, loss function, 
# model path, current maximum corretly labeled predictions in validation loader
# returns model, new maximum corretly labeled predictions in validation loader
def train_model(model,epochs, trainloader,validloader, optimizer, criterion, device, model_path_name,valid_count_max):

    start=time.time()
    print (datetime.datetime.now())
    steps = 0
    print_every = 20
    #test_acc_max = 0.8*len(testloader) # track change in validation loss
    for epoch in range(epochs):
        step=0
        train_loss = 0.0
        model.train()
        for inputs, labels in trainloader:
            step += 1
            if(step%print_every==0):
                print(f"Epoch {epoch+1}/{epochs}..step {step}/{len(trainloader)}.."
                      f"time so far {(time.time()-start)/60:.3f} mins")
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
        train_loss=train_loss/len(trainloader)
        valid_accuracy, valid_loss,valid_count=test_model(model,validloader,criterion,device)
        print (datetime.datetime.now())
        print(f"Epoch {epoch+1}/{epochs}.. "
              f"Train loss: {train_loss:.3f}.. "
              f"Valid loss: {valid_loss:.3f}.. "
              f"Valid accuracy: {valid_accuracy:.5f}.."
              f"Time Elspased so far: {time.time()-start:.3f} seconds")
        train_loss = 0
        train_accuracy=0
        if valid_count>valid_count_max:
            print(f"*************New Model Saved with correct predictions of:{valid_count:.5f}***************")
            torch.save(model, model_path_name)
            valid_count_max= valid_count
    return model,valid_count_max

## 4. Model Creation

In his section, the neural network is created by either loaded from models library or loaded from pre-saved model file. The FC classifier needs to be changed to fit the problem requirements. 

In [ ]:
# Model instainstiation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if(retrain):
    model = torch.load(model_path)
else:
    model=eval("models."+selected_net+"(pretrained=net_pretrained)")
    torch.save(model, model_path)
    if(not full_train):
        for par in  model.parameters():
            param.requires_grad=False
    #change FC classifier
    # You may change this part according to the the problem requirments
    model.fc = nn.Sequential(nn.Linear(FC[net_index], 1024),
                                     nn.Tanh(),
                                     nn.Linear(1024, 256),
                                     nn.Tanh(),
                                     #nn.Dropout(0.2), #optional
                                     nn.Linear(256, output_classes),
                                     nn.LogSoftmax(dim=1)) 
    

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.to(device);

In [ ]:
print(device)
print(model)

## 5. Model Training

### 5.1 Model Current  Performance

In [ ]:
# test model intially over validation set
valid_acc_max,valid_loss,valid_count_max=test_model(model,validloader,criterion,device)

In [ ]:
torch.save(model,model_path)
valid_acc_max,valid_loss,valid_count_max

## 5.2 Model Training

In [ ]:
#train model
model,valid_count_max = train_model(model,2, trainloader,validloader, optimizer,criterion,device,model_path,valid_count_max) 
print(valid_count_max)

# 6. Model Testing

In [ ]:
#load best model
model = torch.load(model_path)
model.to(device)
acc,_,report,conf_matrix=test_report(model,testloader,device)
print("Test Data Set Accuracy = ",acc)
print("Calssification Report")
print("*"*50)
print(report)

In [ ]:
#view Confusion Matrix
df=pd.DataFrame(conf_matrix)
pd.set_option('display.max_columns', output_classes,'display.max_rows',output_classes)
df

In [ ]:
loader_dic={"train":trainloader,"valid":validloader,"test":testloader}
#loader_dic={"valid":validloader,"test":testloader}
for i in loader_dic:
    loader=loader_dic[i]
    print(i+" data set")
    test_report_model(model,loader,device)
    print("*"*50)